In [ ]:
import os
import gc
import numpy as np
import librosa.display
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model, Sequential, load_model
from tensorflow.keras.applications import MobileNet, MobileNetV3Small, MobileNetV3Large, VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input, decode_predictions
from tensorflow.keras.layers import (
    Conv2D, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D, BatchNormalization
)
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, classification_report, accuracy_score, roc_curve


2025-03-08 19:04:43.637371: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-08 19:04:43.647682: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741435483.658669  411149 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741435483.661897  411149 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-08 19:04:43.675207: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [ ]:
N_MFCC = 128 

In [ ]:
X = np.load('../SavedFeatures/X_mfcc.npy')
y = np.load('../SavedFeatures/y_mfcc.npy')


X_train, X_val, y_train, y_val= train_test_split(X, y, test_size = 0.2, random_state = 42)

y_train = to_categorical(y_train, num_classes=2)
y_val = to_categorical(y_val, num_classes=2)

del X, y  
gc.collect()

In [11]:
def model_vgg16(input_shape=(N_MFCC, 109, 1), num_classes=2):
    base_model = VGG16(weights= None, include_top=False, input_shape=input_shape)
    
    x = GlobalAveragePooling2D()(base_model.output)
    x = Dense(512, activation='relu')(x)
    x = BatchNormalization()(x)  
    x = Dropout(0.5)(x)
    
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.4)(x)
    
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.3)(x)
   
    x = Dense(num_classes, activation='softmax')(x)
    return Model(inputs=base_model.input, outputs=x)

In [12]:
mvgg16 = model_vgg16(input_shape=(N_MFCC, 109, 1), num_classes=2)
mvgg16.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 128, 109, 1)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 128, 109, 64)   │           640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 128, 109, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 64, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 64, 54, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 64, 54, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 32, 27, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 32, 27, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 32, 27, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 32, 27, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 16, 13, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 16, 13, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 16, 13, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 16, 13, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 8, 6, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 8, 6, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 8, 6, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 8, 6, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 4, 3, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 512)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │       262,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 512)            │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 256)            │       131,32

 Total params: 15,142,722 (57.76 MB)

 Trainable params: 15,141,698 (57.76 MB)

 Non-trainable params: 1,024 (4.00 KB)

In [ ]:
mvgg16.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
                  loss='binary_crossentropy',  
                  metrics=['accuracy'])

In [ ]:
tf.keras.backend.clear_session()

In [ ]:
hist = mvgg16.fit(X_train, y_train, batch_size=16, epochs=10, validation_data=(X_val, y_val))

Epoch 1/10
637/637 ━━━━━━━━━━━━━━━━━━━━ 36s 49ms/step - accuracy: 0.8858 - loss: 0.2887 - val_accuracy: 0.4013 - val_loss: 11.1319
Epoch 2/10
637/637 ━━━━━━━━━━━━━━━━━━━━ 27s 42ms/step - accuracy: 0.9640 - loss: 0.1086 - val_accuracy: 0.6753 - val_loss: 1.8375
Epoch 3/10
637/637 ━━━━━━━━━━━━━━━━━━━━ 27s 42ms/step - accuracy: 0.9722 - loss: 0.0937 - val_accuracy: 0.9639 - val_loss: 0.1032
Epoch 4/10
637/637 ━━━━━━━━━━━━━━━━━━━━ 27s 42ms/step - accuracy: 0.9752 - loss: 0.0774 - val_accuracy: 0.9733 - val_loss: 0.0762
Epoch 5/10
637/637 ━━━━━━━━━━━━━━━━━━━━ 27s 42ms/step - accuracy: 0.9781 - loss: 0.0782 - val_accuracy: 0.9753 - val_loss: 0.0740
Epoch 6/10
637/637 ━━━━━━━━━━━━━━━━━━━━ 27s 42ms/step - accuracy: 0.9881 - loss: 0.0388 - val_accuracy: 0.5163 - val_loss: 1.8315
Epoch 7/10
637/637 ━━━━━━━━━━━━━━━━━━━━ 27s 42ms/step - accuracy: 0.9875 - loss: 0.0382 - val_accuracy: 0.6046 - val_loss: 2.8631
Epoch 8/10
637/637 ━━━━━━━━━━━━━━━━━━━━ 27s 42ms/step - accuracy: 0.9819 - loss: 0.0515 -

In [16]:
test_loss, test_accuracy = mvgg16.evaluate(X_val, y_val, verbose=0)
y_pred = mvgg16.predict(X_val)
y_pred_classes = y_pred.argmax(axis=1) 
y_true_classes = y_val


80/80 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step


In [ ]:
y_pred_classes = y_pred.argmax(axis=1) 
y_true_classes = y_val.argmax(axis=1) 

f1 = f1_score(y_true_classes, y_pred_classes, average='weighted')

print("===================VGG16 - MS===================")
print(f"TAccuracy: {test_accuracy:.5f}")
print(f"F1-Score: {f1:.5f}")

# Tính EER
eers = []


for i in range(y_pred.shape[1]): 
    
    y_true_binary = y_val[:, i]
    y_pred_prob = y_pred[:, i]

    
    fpr, tpr, thresholds = roc_curve(y_true_binary, y_pred_prob)
    fnr = 1 - tpr
    
    
    eer_threshold = thresholds[np.nanargmin(np.abs(fpr - fnr))]
    eer = fpr[np.nanargmin(np.abs(fpr - fnr))]
    eers.append(eer)
    print(f"Class {i}: EER = {eer:.5f} at threshold {eer_threshold:.5f}")


mean_eer = np.mean(eers)
print(f"EER: {mean_eer:.5f}")


===================VGG16 - MS===================
TAccuracy: 0.89360
F1-Score: 0.89464
Class 0: EER = 0.00489 at threshold 0.00010
Class 1: EER = 0.00459 at threshold 0.99991
EER: 0.00474


===========

In [19]:
# Save model as weights in H5 format
mvgg16.save_weights("../SavedModels/MFCC_mvgg16.weights.h5")

In [ ]:
del mvgg16, X_train, X_val, y_train, y_val
gc.collect()